In [1]:
from IPython.display import display_html
display_html(
    """<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>""", raw=True)

In [2]:
import xml.etree.ElementTree as ET
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [4]:
def genCreateSQL(filePath):
    DOMTree = ET.parse(filePath)
    #获取表名
    tblName=DOMTree.findall('./Method/Name')[0].text
    if tblName[5] == '_':
        tblName = tblName[6:]
    else:
        tblName = tblName[5:]
    #获取备注信息
    tblComment=DOMTree.findall('./Method/Items/Item/Value/Data')[4].text

    #获取字段信息
    fields=DOMTree.findall('./Method/Items/Item/Value')[6]

    primary_key = 'ID'
    result = []
    result.append('CREATE TABLE {} (\n'.format(tblName))
    for field in fields.findall('Fields/Field'):
        colName = field.text
        colType = field.get("type")
        colLength = field.get("length")
        colComment = field.get("CHNAME")
        if  colType == 'string':
            colType = 'VARCHAR({})'.format(colLength)
        colDefine =  "{} {} COMMENT '{}',\n".format(colName,colType,colComment)
        result.append(colDefine)
    result.append('PRIMARY KEY ({})\n'.format(primary_key))
    result.append(") COMMENT '{}';\n".format(tblComment))
    return tblName,''.join(result)

def genColInfos(filePath):
    DOMTree = ET.parse(filePath)

    #获取字段信息
    fields=DOMTree.findall('./Method/Items/Item/Value')[6]

    result = []
    result.append("{}\t{}\t{}\t{}\t{}\n".format('colName','colType','colComment','colLength','scale'))
    for field in fields.findall('Fields/Field'):
        colName = field.text
        colType = field.get("type")
        colLength = field.get("length")
        colComment = field.get("CHNAME")
        colDefine =  "{}\t{}\t{}\t{}\t{}\n".format(colName,colType,colComment,colLength,field.get("scale"))
        result.append(colDefine)
    return ''.join(result)

def genColTypes(filePath):
    DOMTree = ET.parse(filePath)

    #获取字段信息
    fields=DOMTree.findall('./Method/Items/Item/Value')[6]

    colTypes = {}
    for field in fields.findall('Fields/Field'):
       colTypes[field.text]=field.get('type')
    return colTypes

def saveFile(file, content, encode='utf-8'):
    """
     数据保存至文件
    :param file: 文件名
    :param dataList: 数据列表
    :param encode: 编码 默认 utf-8
    :return: 无返回
    """
    f = open(file=file, mode='w+', encoding=encode)
    f.write(content)
    f.close()
    
import re

def genSampleData(dataFilePath,defFilePath):
    colTypes=genColTypes(defFilePath)
    colTypes=list(colTypes.values())
    DOMTree = ET.parse(dataFilePath)

    #获取字段信息
    colNames=DOMTree.findall('./Method/Items/Item/Value')[0]
    rows=DOMTree.findall('./Method/Items/Item/Value')[1]
    columns=[]
    for colName in colNames.findall('Data'):
        columns.append(str(colName.text).strip())        

    result = []
    for record in rows.findall('Records/Record'):
        row=[]
        idx=0
        for data in record.findall('Data'):
            if colTypes[idx] == 'string':
                row.append(re.sub('\s+',' ',str(data.text).strip()))
            else:
                row.append(data.text)
            idx+=1
        result.append(row)
    
    df = pd.DataFrame(result, columns=columns[0:len(result[0])])
    return df

In [8]:
tblName,sql=genCreateSQL('data/面单记录结构表.xml')
print(tblName,sql,sep='\n')

PEXPRESS
CREATE TABLE PEXPRESS (
GOODS VARCHAR(500) COMMENT '物品名',
S_COUNTRY VARCHAR(128) COMMENT '寄件人国家',
S_CSN VARCHAR(64) COMMENT '寄件人国籍识别码',
S_HOUSEHOLDS VARCHAR(500) COMMENT '寄件人户籍地址',
S_PROVINCE VARCHAR(128) COMMENT '寄件人省',
S_CITY VARCHAR(128) COMMENT '寄件人市',
S_NATION VARCHAR(128) COMMENT '寄件人民族',
S_SEX VARCHAR(64) COMMENT '寄件人性别',
G_COUNTRY VARCHAR(128) COMMENT '收件人国籍',
G_CSN VARCHAR(64) COMMENT '收件人国籍识别码',
G_HOUSEHOLDS VARCHAR(500) COMMENT '收件人户籍地址',
G_PROVINCE VARCHAR(128) COMMENT '收件人省',
G_CITY VARCHAR(128) COMMENT '收件人市',
G_NATION VARCHAR(128) COMMENT '收件人民族',
G_SEX VARCHAR(64) COMMENT '收件人性别',
PROVINCE VARCHAR(128) COMMENT '企业所在省',
CITY VARCHAR(128) COMMENT '企业所在市',
DISTRICT VARCHAR(128) COMMENT '企业所在区',
G_GPSADDRESS VARCHAR(500) COMMENT '收件人地址',
G_SID VARCHAR(500) COMMENT '收件人身份证',
G_ADDRESS VARCHAR(500) COMMENT '收件人地址',
CREATEDTIME date COMMENT '创建时间',
GZDB_ADDTIME date COMMENT '数据入库时间',
ITEMID VARCHAR(128) COMMENT '主键',
S_NAME VARCHAR(500) COMMENT '发件人姓名',
S_SID VARCHAR(

In [9]:
colInfos=genColInfos('data/面单记录结构表.xml')
print(colInfos)

colName	colType	colComment	colLength	scale
GOODS	string	物品名	500	0
S_COUNTRY	string	寄件人国家	128	0
S_CSN	string	寄件人国籍识别码	64	0
S_HOUSEHOLDS	string	寄件人户籍地址	500	0
S_PROVINCE	string	寄件人省	128	0
S_CITY	string	寄件人市	128	0
S_NATION	string	寄件人民族	128	0
S_SEX	string	寄件人性别	64	0
G_COUNTRY	string	收件人国籍	128	0
G_CSN	string	收件人国籍识别码	64	0
G_HOUSEHOLDS	string	收件人户籍地址	500	0
G_PROVINCE	string	收件人省	128	0
G_CITY	string	收件人市	128	0
G_NATION	string	收件人民族	128	0
G_SEX	string	收件人性别	64	0
PROVINCE	string	企业所在省	128	0
CITY	string	企业所在市	128	0
DISTRICT	string	企业所在区	128	0
G_GPSADDRESS	string	收件人地址	500	0
G_SID	string	收件人身份证	500	0
G_ADDRESS	string	收件人地址	500	0
CREATEDTIME	date	创建时间	0	0
GZDB_ADDTIME	date	数据入库时间	0	0
ITEMID	string	主键	128	0
S_NAME	string	发件人姓名	500	0
S_SID	string	发件人证件号码	500	0
S_TEL	string	发件人手机号码	500	0
S_ADDRESS	string	发件人地址	500	0
S_LAT	double	发件人x（精度）坐标	38	15
S_LON	double	发件人y（纬度）坐标	38	15
G_NAME	string	收件人姓名	500	0
GCOUNT	string	物品数量	500	0
EXPRESS_SN	string	单号	500	0
COID	string	公司id	500	0
CONAME	string	公司名	500	0
COA

In [11]:
colTypes=genColTypes('data/面单记录结构表.xml')
print(colTypes)

{'GOODS': 'string', 'S_COUNTRY': 'string', 'S_CSN': 'string', 'S_HOUSEHOLDS': 'string', 'S_PROVINCE': 'string', 'S_CITY': 'string', 'S_NATION': 'string', 'S_SEX': 'string', 'G_COUNTRY': 'string', 'G_CSN': 'string', 'G_HOUSEHOLDS': 'string', 'G_PROVINCE': 'string', 'G_CITY': 'string', 'G_NATION': 'string', 'G_SEX': 'string', 'PROVINCE': 'string', 'CITY': 'string', 'DISTRICT': 'string', 'G_GPSADDRESS': 'string', 'G_SID': 'string', 'G_ADDRESS': 'string', 'CREATEDTIME': 'date', 'GZDB_ADDTIME': 'date', 'ITEMID': 'string', 'S_NAME': 'string', 'S_SID': 'string', 'S_TEL': 'string', 'S_ADDRESS': 'string', 'S_LAT': 'double', 'S_LON': 'double', 'G_NAME': 'string', 'GCOUNT': 'string', 'EXPRESS_SN': 'string', 'COID': 'string', 'CONAME': 'string', 'COADDRESS': 'string', 'EBOYID': 'string', 'G_LON': 'double', 'EBOY': 'string', 'EBOYTEL': 'string', 'OPENPIC': 'binary', 'TDAY': 'date', 'CJSJ': 'date', 'RKSJ': 'date', 'STREE_ID': 'string', 'POLCE_STATION_ID': 'string', 'COMMUNITY_ID': 'string', 'G_TEL':

In [14]:
df=genSampleData('data/面单记录表.xml','data/面单记录结构表.xml')   
df

,GOODS,S_COUNTRY,S_CSN,S_HOUSEHOLDS,S_PROVINCE,S_CITY,S_NATION,S_SEX,G_COUNTRY,G_CSN,...,EBOYTEL,OPENPIC,TDAY,CJSJ,RKSJ,STREE_ID,POLCE_STATION_ID,COMMUNITY_ID,G_TEL,G_LAT
0,文具,null,null,null,null,null,null,null,null,null,...,13588888888,null,2017-09-12\n 00...,2017-09-12 08:48:54.0,2017-09-12\n 08...,440111003,440111570000,440111003010,13907918121,0


In [16]:
df.iloc[0]

GOODS                                                              文具
S_COUNTRY                                                        null
S_CSN                                                            null
S_HOUSEHOLDS                                                     null
S_PROVINCE                                                       null
S_CITY                                                           null
S_NATION                                                         null
S_SEX                                                            null
G_COUNTRY                                                        null
G_CSN                                                            null
G_HOUSEHOLDS                                                     null
G_PROVINCE                                                       null
G_CITY                                                           null
G_NATION                                                         null
G_SEX               